In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/run/media/alf/datos/misrepositorios/docencia/analisis-practicas-julia"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using SymPy  # Para el cálculo simbólico de límites.
using Plots  # Para el dibujo de gráficas.
#plotlyjs() # Para obtener gráficos interactivos.
using LaTeXStrings  # Para usar código LaTeX en los gráficos.

In [3]:
f(x) = x^2
Δx = 1/10
areas_inf = [f((i-1)*Δx)*Δx for i = 1:10]
sum_inf = sum(areas_inf)

0.2850000000000001

In [4]:
f(x) = x^2
Δx = 1/10
areas_sup = [f(i*Δx)*Δx for i = 1:10]
sum_sup = sum(areas_sup)

0.3850000000000001

In [5]:
error = sum_sup - sum_inf

0.10000000000000003

In [6]:
using PrettyTables
f(x) = x^2
function area_inf(a, b, n)
    Δx = (b-a)/n
    return sum([f(a+(i-1)*Δx)*Δx for i = 1:n])
end

function area_sup(a, b, n)
    Δx = (b-a)/n
    return sum([f(a+i*Δx)*Δx for i = 1:n])
end

function area(a, b, n)
    area = (area_inf(a, b, n) + area_sup(a, b, n)) / 2
    error = area_sup(a, b, n) - area_inf(a, b, n)
    return area, error
end

areas = [area(0, 1, n) for n=10:100]
pretty_table(hcat(first.(areas), last.(areas)); header = ["Aproximación", "Error"])

┌──────────────┬───────────┐
│ Aproximación │     Error │
├──────────────┼───────────┤
│        0.335 │       0.1 │
│     0.334711 │ 0.0909091 │
│     0.334491 │ 0.0833333 │
│      0.33432 │ 0.0769231 │
│     0.334184 │ 0.0714286 │
│     0.334074 │ 0.0666667 │
│     0.333984 │    0.0625 │
│      0.33391 │ 0.0588235 │
│     0.333848 │ 0.0555556 │
│     0.333795 │ 0.0526316 │
│      0.33375 │      0.05 │
│     0.333711 │  0.047619 │
│     0.333678 │ 0.0454545 │
│     0.333648 │ 0.0434783 │
│     0.333623 │ 0.0416667 │
│       0.3336 │      0.04 │
│      0.33358 │ 0.0384615 │
│     0.333562 │  0.037037 │
│     0.333546 │ 0.0357143 │
│     0.333532 │ 0.0344828 │
│     0.333519 │ 0.0333333 │
│     0.333507 │ 0.0322581 │
│     0.333496 │   0.03125 │
│     0.333486 │  0.030303 │
│     0.333478 │ 0.0294118 │
│     0.333469 │ 0.0285714 │
│     0.333462 │ 0.0277778 │
│     0.333455 │  0.027027 │
│     0.333449 │ 0.0263158 │
│     0.333443 │  0.025641 │
│     0.333438 │     0.025 │
│     0.333432

In [7]:
using SymPy
@vars x real=true
integrate(x^2*ln(x))

 3           3
x ⋅log(x)   x 
───────── - ──
    3       9 

In [8]:
integrate(ln(ln(x))/x)

log(x)⋅log(log(x)) - log(x)

In [9]:
integrate((5x^2+4x+1)/(x^5-2x^4+2x^3-2x^2+x))

                           ⎛ 2    ⎞                 5  
log(x) - 3⋅log(x - 1) + log⎝x  + 1⎠ - 2⋅atan(x) - ─────
                                                  x - 1

In [10]:
integrate((6x+5)/(x^2+x+1)^2)

                          ⎛2⋅√3⋅x   √3⎞
                 8⋅√3⋅atan⎜────── + ──⎟
   4⋅x - 7                ⎝  3      3 ⎠
────────────── + ──────────────────────
   2                       9           
3⋅x  + 3⋅x + 3                         

In [11]:
using SymPy
@vars x real=true
integrate(x^3/x^2+x+1, -1/2, 0)

0.250000000000000

In [12]:
integrate(sqrt(16-x^2)/x, 2, 4)

-2⋅√3 + 4⋅acosh(2)

In [13]:
integrate(1/(3+cos(2x)), 0, PI/2)

√2⋅π
────
 8  

In [14]:
N = 50
a(n) = 1/2^n
an = [a(n) for n = 1:N]
An = cumsum(an)

50-element Vector{Float64}:
 0.5
 0.75
 0.875
 0.9375
 0.96875
 0.984375
 0.9921875
 0.99609375
 0.998046875
 0.9990234375
 0.99951171875
 0.999755859375
 0.9998779296875
 ⋮
 0.999999999998181
 0.9999999999990905
 0.9999999999995453
 0.9999999999997726
 0.9999999999998863
 0.9999999999999432
 0.9999999999999716
 0.9999999999999858
 0.9999999999999929
 0.9999999999999964
 0.9999999999999982
 0.9999999999999991

In [15]:
N = 50
a(n) = 1/2^n
A(n) = sum(a, 1:n)
An = [A(n) for n = 1:N]

50-element Vector{Float64}:
 0.5
 0.75
 0.875
 0.9375
 0.96875
 0.984375
 0.9921875
 0.99609375
 0.998046875
 0.9990234375
 0.99951171875
 0.999755859375
 0.9998779296875
 ⋮
 0.999999999998181
 0.9999999999990905
 0.9999999999995453
 0.9999999999997726
 0.9999999999998863
 0.9999999999999432
 0.9999999999999716
 0.9999999999999858
 0.9999999999999929
 0.9999999999999964
 0.9999999999999982
 0.9999999999999991

In [16]:
#| echo: false
using QuizQuestions
nothing

In [17]:
#| echo: false
a(n) = 1/(big(n)*2^big(n))
numericq(sum(a, 1:50), 1e-20, hint="Introducir hasta 20 decimales")

QuizQuestions.Numericq(0.6931471805599452923248950070631627496611873046174183751999313065846593137772644, 1.0e-20, "", "", "Introducir hasta 20 decimales", "", nothing)

In [18]:
#| echo: false
a(n) = factorial(big(n))/big(n)^big(n)
numericq(sum(a, 1:100), 1e-40, hint="Introducir hasta 40 decimales")

QuizQuestions.Numericq(1.879853862175258533486306145070960038819872853086289619594759148401709391879149, 1.0e-40, "", "", "Introducir hasta 40 decimales", "", nothing)